In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [3]:
df = pd.read_csv(filepath_or_buffer='./analysis_df.csv', delimiter=',')
emp_df = pd.read_csv(filepath_or_buffer='./analysis_df_employee.csv', delimiter=',')

In [4]:
df.head(5)

,Age,Gender,Salary,Title,Formal Education,Learning Platforms,Coding Experience,ML Experience,Important Activities,Languages,...,Published Papers,ML Research,ML Serve,Ethical AI tools,Industry Type,Data Storage Products,Data Products,ML Monitor Tools,Country,Year
0,18-21,Man,1528.0,Student,Bachelor's degree,Coursera -- -1,< 1 year,< 1 year,Build and/or run a machine learning service th...,Python -- Java -- C/C++ -- -1,...,NaN,NaN,NaN,NaN,I am a student,NaN,NaN,NaN,Viet Nam,2018
1,18-21,Man,1528.0,Student,Some college/university study without<br>earni...,Udacity -- Coursera -- edX -- Udemy -- -1,< 1 year,< 1 year,Analyze and understand data to influence produ...,Python -- -1,...,NaN,NaN,NaN,NaN,Computers/Technology,NaN,NaN,NaN,Viet Nam,2018
2,18-21,Man,NaN,Student,Bachelor's degree,Coursera -- -1,1-5 years,1-2 years,Build and/or run a machine learning service th...,Python -- -1,...,NaN,NaN,NaN,NaN,Computers/Technology,NaN,NaN,NaN,Viet Nam,2018
3,22-24,Man,1528.0,Student,Master's degree,Coursera -- DataQuest -- -1,< 1 year,< 1 year,Do research that advances the state of the art...,Python -- R -- Bash -- -1,...,NaN,NaN,NaN,NaN,I am a student,NaN,NaN,NaN,Viet Nam,2018
4,22-24,Man,1528.0,Data Scientist,Bachelor's degree,Coursera -- -1,1-5 years,< 1 year,Analyze and understand data to influence produ...,R -- SQL -- -1,...,NaN,NaN,NaN,NaN,Insurance/Risk Assessment,NaN,NaN,NaN,Viet Nam,2018


# Plot 1:

In [26]:
coding_exp_counts = df['Coding Experience'].value_counts()
avg_salary = df.groupby('Coding Experience')['Salary'].mean().sort_values()

fig = sp.make_subplots(
    rows=1, cols=2,
    column_widths=[0.5, 0.2],
    row_heights=[2],
    subplot_titles=('Average Salary by Coding Experience', 'Coding Experience Distribution'),
    specs=[[{"type": "bar"}, {"type": "pie"}]])

fig.add_trace(go.Bar(x=avg_salary.index, y=avg_salary.values, name='avg_salary', marker_color='red'), row=1, col=1)

fig.add_trace(go.Pie(labels=coding_exp_counts.index, values=coding_exp_counts.values, name='coding_exp'), row=1, col=2)

fig.update_layout(
    title='Coding Experience and Salary',
    xaxis_title="Experience",
    yaxis_title="salary",
    grid=dict(rows=1, columns=2),
    legend_title="Coding Experience",
    template='plotly_white'
)

fig.show()

Nhận xét:
- Với biểu đồ cột, 1 điều khá hiển nhiên là kinh nghiệm càng nhiều thì mức lương càng cao
- Mức lương trung bình của nhân viên chưa bao giờ viết code cao hơn so với mức lương trung bình của nhân viên có dưới 1 năm kinh nghiêm lập trình, điều này có thể được giải thích với biểu đồ tròn, khi 
 tỉ lệ nhân viên có kinh nghiệm dưới 1 năm (chỉ thua 1-5 năm) nhiều hơn đáng kể so với tỉ lệ nhân viên chưa bao giờ lập trình.
- Tỉ lệ nhân viên có kinh nghiệm lập trình từ 1-5 năm là cao nhất, trong khi ngược lại, tỉ lệ nhân viên có kinh nghiệm lập trình hơn 20 năm là thấp nhất

# Plot 2:

In [24]:
unwanted = {'Currently not employed', 'Not employed', 'Student'}
title_list = df['Title'].value_counts().nlargest(n=14).index.to_list()
title_list = [e for e in title_list if e not in unwanted]

rate_df = df[df['Title'].isin(title_list)]
rate_df = rate_df.groupby(['Title', 'Gender']).size() / df.groupby('Title').size() * 100
rate_df = rate_df.reset_index(name='Rate')
rate_df = rate_df.sort_values(by='Rate')

rate_df['Gender'] = rate_df['Gender'].replace(['Nonbinary', 'Prefer to self-describe', 'Prefer not to say'], 'Others')

fig = go.Figure()

for gender in rate_df['Gender'].unique():
    fig.add_trace(
        go.Bar(
            x=rate_df[rate_df['Gender'] == gender]['Title'],
            y=rate_df[rate_df['Gender'] == gender]['Rate'],
            name=gender
        )
    )

fig.update_layout(
    title={
        'text': 'Gender Rate by Occupation',
        'x':0.5,
        'y': 0.96
    },
    xaxis_title='Occupation',
    yaxis_title='Rate (%)',
    template='plotly_white',
    barmode='group',
    legend=dict(
        orientation="h",
        yanchor="middle",
        y=1.05,
        xanchor="center",
        x=0.95
    ),
    bargap=0.2,
    autosize=False,
    width=1250,
    height=600,
    margin=dict(l=50, r=50, t=50, b=50),
    xaxis=dict(type='category'),
    hovermode='x'
)

fig.show()

Nhận xét:
- Trong cả top 10 ngành nghề hot nhất, tỉ lệ nhân viên nam đều vượt trội hơn nhiều so với tỉ lệ nhân viên nữ (trung bình gấp ít nhất 5 lần)
- Tuy nhiên, đối với công việc giáo viên/giáo sư (Teacher/Professor), tỉ lệ nam nữ khá cân đối khi tỉ lệ giáo viên nữ chỉ thua giáo viên nam khoảng hơn 10%
- Manager là lĩnh vực công việc mà tỉ lệ nam vượt trội nhất so với nữ

# Plot 3:

In [57]:
title_counts = df['Title'].value_counts().reset_index()
title_counts.columns = ['Title', 'Distribution']

avg_salary = df.groupby('Title')['Salary'].mean().reset_index()

color_palette = ['#FF5733', '#FFC300', '#900C3F', '#008080', '#2ECC71', '#3498DB', '#9B59B6', '#F1948A',
                 '#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF', '#C0C0C0', '#808080',
                 '#800000', '#808000', '#008000', '#800080', '#008080', '#000080', '#FFFFF0', '#F0FFF0',
                 '#F0FFFF', '#FFFAF0', '#F5F5DC', '#FAEBD7', '#FFEFD5', '#FFE4B5', '#FFE4C4']

hover_text = [
    f"{title_counts['Title'][i]}<br>Average Salary: {avg_salary['Salary'][i]:.2f}K<br>Distribution: {title_counts['Distribution'][i]}"
    for i in range(len(title_counts))
]

fig_scatter = go.Figure(data=go.Scatter(
    x=title_counts['Distribution'],
    y=avg_salary['Salary'],
    mode='markers',
    text=hover_text,
    hoverinfo='text',
    marker=dict(
        size=10,
        color=color_palette,
        opacity=0.8,
        line=dict(width=0.5, color='black')
    )
))

fig_scatter.update_layout(
    title='Job Title Distribution vs Average Salary',
    xaxis_title='Distribution',
    yaxis_title='Average Salary',
    hovermode='closest',
)

fig_scatter.show()

Nhận xét:
- Sinh viên có số lượng cao nhất trong tất cả các ngành nghề, trong khi số lượng nhà báo dữ liệu thì thấp nhất.
- Kỹ sư Máy học (Machine Learning Engineer) có mức lương trung bình cao nhất, trong khi quản trị viên dữ liệu (Data Administrator) có mức lương trung bình thấp nhất.
- Nhà khoa học dữ liệu không chỉ là một trong những ngành nghề phổ biến nhất mà còn cung cấp mức lương rất cao.